In [ ]:
from io import BytesIO
import os
from PIL import Image, ExifTags
from tqdm import tqdm

source_dir = ('/content/drive/MyDrive/Colab Notebooks/snap-sorter/Batchsnaps’24/Day 2')
files = []

for dirpath, dirnames, filenames in os.walk(source_dir):
    for filename in filenames:
        if filename.lower().endswith('.jpg'):
            full_file_name = os.path.join(dirpath, filename)
            if os.path.isfile(full_file_name):
                files.append(full_file_name)
print(len(files))

51


In [ ]:
from concurrent.futures import ThreadPoolExecutor

output_dir = "/content/facess"
os.makedirs(output_dir, exist_ok=True)

def compress_and_save_image(image):
    try:
        img = Image.open(image)
        exif_data = img.info.get("exif")
        buffer = BytesIO()
        img.save(buffer, "JPEG", quality=10, exif=exif_data)
        output_path = os.path.join(output_dir, os.path.splitext(os.path.basename(image))[0] + ".JPG")
        with open(output_path, "wb") as handle:
            handle.write(buffer.getbuffer())

    except Exception as e:
        print(f"Error compressing {image}: {e}")
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(compress_and_save_image, files), desc="Compressing Images", total=len(files)))


Compressing Images: 100%|██████████| 51/51 [00:32<00:00,  1.55it/s]


In [ ]:
import zipfile

def zip_folder_with_progress(folder_path, output_zip):
    total_size = sum(os.path.getsize(os.path.join(root, file))
                     for root, dirs, files in os.walk(folder_path)
                     for file in files)

    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        with tqdm(total=total_size, unit='B', unit_scale=True, desc="Zipping") as pbar:
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, os.path.relpath(file_path, folder_path))
                    pbar.update(os.path.getsize(file_path))

folder_path = '/content/facess'
output_zip = '/content/output.zip'
zip_folder_with_progress(folder_path, output_zip)

Zipping: 100%|██████████| 39.4M/39.4M [00:03<00:00, 10.6MB/s]
